## Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import time

In [3]:
from pysmFISH.pipeline import Pipeline

/home/simone/mini/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/home/simone/mini/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/home/simone/mini/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


## Initialize the pipeline

In [4]:
# Enter the required parameters
experiment_fpath = Path('/datc/sl/fish_rawdata/AMEXP20220301_EEL_SNRN3')
dataset_name = '220308_21_06_09_AMEXP20220301_EEL_SNRN3_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [5]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        chunk_size = 10,
        scheduler_port = 237275,
        dataset_path = experiment_fpath / dataset_name)

CPU times: user 188 µs, sys: 0 ns, total: 188 µs
Wall time: 197 µs


## Run the required steps

In [6]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 23122 instead
distributed.deploy.ssh - INFO - warnings.warn(
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -------------------------

CPU times: user 6.01 s, sys: 3.55 s, total: 9.56 s
Wall time: 1min 24s


In [18]:
running_pipeline.client

<Client: 'tcp://193.10.16.58:36867' processes=160 threads=160, memory=29.10 TiB>

## Process the images of the fresh tissue

In [8]:
%%time
running_pipeline.processing_fresh_tissue_step()

KilledWorker: ('stitch_using_coords_general-b71e22c37cf2fd87f57d08ef29b802d1', <WorkerState 'tcp://192.168.0.12:43906', name: 2-39, memory: 0, processing: 16>)

In [19]:
running_pipeline.client.close()

In [20]:
running_pipeline.cluster.close()